### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle
from countpassenger import Dataset

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-11"
SAVE_DIR = os.path.join(
    conf.RESOURCES_PROCESSED_DIR, f"2024-04-29/count-passengers-{CURRENT_CAM}-vehicle.csv"
)
load_param = dict(
    vehicle_path_rel="2024-04-29/mbk-tourist-vehicle-object-20240429.csv",
    cross_path_rel="2024-04-29/mbk-tourist-raw-cross-object-20240429.csv",
    reverse_path_rel="2024-04-29/mbk-tourist-raw-reverse-object-20240429.csv",
)

In [2]:
df_vehicle, df_cross, df_reverse = Dataset.load_dataset_from_paths(**load_param)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

## Cross/Reverse

In [3]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

In [4]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [5]:
df_reverse_with_reverse_count = match_reverse_to_vehicle(df_reverse, df_vehicle)
df_vehicle_with_cross_and_reverse = match_cross_to_vehicle(
    df_cross,
    df_reverse_with_reverse_count,
)

# df_vehicle = df_vehicle[
#     (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
# ]
print(
    df_vehicle_with_cross_and_reverse[
        [
            *conf.VEHICLE_INTEREST_NON_SNAPSHOT,
            "cross_count",
            "reverse_count",
            # "frame_no",
        ]
    ].to_string()
)

                 camera       timestamp_precise  lifetime    xmid   ymid plate_number vehicle_type  timestamp_unix  timestamp_unix_end  cross_count  reverse_count
1563  mbk-14-11-vehicle 2024-04-29 02:09:10.575     2.599  1377.0  118.0       กก4435       pickup      1714356550          1714356552            0              0
1551  mbk-14-11-vehicle 2024-04-29 02:10:17.760     3.479  1378.5  179.0      1กท9770          suv      1714356617          1714356620            0              0
1545  mbk-14-11-vehicle 2024-04-29 02:24:51.844     1.399  1365.5  108.5          NaN          NaN      1714357491          1714357492            0              0
1544  mbk-14-11-vehicle 2024-04-29 02:28:48.181    31.711  1870.0  213.0      6กจ2543          suv      1714357728          1714357759            2              1
1540  mbk-14-11-vehicle 2024-04-29 02:30:57.229    51.625  1909.0  227.5      1มข4258         taxi      1714357857          1714357908            0              0
1541  mbk-14-11-vehicl

In [6]:
# df_vehicle_with_cross_and_reverse.to_csv(path_or_buf=SAVE_DIR, index=False)

## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?